In [ ]:
!pip install -q transformers datasets accelerate bitsandbytes peft trl

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
print(f"GPU Memory available: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")


In [ ]:
# Step 3: Import libraries
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import os

In [ ]:
print("="*60)
print("🚀 SmolLM3-3B Email Fine-tuning (4-bit QLoRA)")
print("="*60)
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print("="*60 + "\n")

In [ ]:
# Step 4: Config
MODEL_NAME = "HuggingFaceTB/SmolLM3-3B"
OUTPUT_DIR = "/kaggle/working/smollm2-email-finetuned"

# Step 5: 4-bit Quantization (QLoRA) - Best for memory!
print("⚙️ Configuring 4-bit quantization (QLoRA)...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

In [ ]:
# Step 6: Load tokenizer
print("📥 Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    use_fast=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
# Step 7: Load model with 4-bit
print("📥 Loading model with 4-bit quantization...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

# Enable gradient checkpointing
model.config.use_cache = False
model.gradient_checkpointing_enable()

print(f"✅ Model loaded! Memory: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB\n")


In [ ]:
# Step 8: LoRA config
print("⚙️ Configuring LoRA...")
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

model.print_trainable_parameters()
print()

In [ ]:
# Step 9: Load email dataset
print("📥 Loading Enron Email dataset...")
dataset = load_dataset("SetFit/enron_spam", split="train")

# Sample only 800 emails
dataset = dataset.shuffle(seed=42).select(range(800))
print(f"Dataset size: {len(dataset)} emails\n")

# Split
train_test = dataset.train_test_split(test_size=0.15, seed=42)
train_dataset = train_test["train"]
eval_dataset = train_test["test"]

In [ ]:
# Step 10: Preprocess
print("⚙️ Preprocessing data...")

def preprocess_function(examples):
    # Get text from dataset
    texts = examples["text"] if "text" in examples else examples["message"]

    # Clean texts
    texts = [f"Email: {str(t).strip()}" for t in texts if t]

    # Tokenize
    result = tokenizer(
        texts,
        truncation=True,
        max_length=128,
        padding="max_length",
    )

    result["labels"] = result["input_ids"].copy()
    return result

train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=train_dataset.column_names,
    desc="Processing train",
)

eval_dataset = eval_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=eval_dataset.column_names,
    desc="Processing eval",
)

print(f"✅ Train: {len(train_dataset)} | Eval: {len(eval_dataset)}\n")

# Clear memory
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# Step 11: Training config
print("⚙️ Setting up training configuration...\n")

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=100,
    save_total_limit=1,
    warmup_steps=30,
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    optim="paged_adamw_32bit",
    gradient_checkpointing=True,
    report_to="none",
    push_to_hub=False,
    dataloader_num_workers=2,
    remove_unused_columns=True,
)


In [ ]:
# Step 12: Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8,
)


In [ ]:
# Step 13: Trainer
print("⚙️ Initializing Trainer...\n")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
)

In [ ]:
# Step 14: Train!
print("="*60)
print("🚀 Starting Training!")
print("="*60)
print(f"Total steps: ~{len(train_dataset) // (4 * 8) * 3}")
print(f"Expected time: ~15-20 minutes")
print("="*60 + "\n")

try:
    trainer.train()
    print("\n" + "="*60)
    print("✅ Training completed successfully!")
    print("="*60 + "\n")

except Exception as e:
    print("\n" + "="*60)
    print(f"⚠️ Training stopped: {str(e)}")
    print("="*60 + "\n")

In [ ]:
# Step 15: Save
print("💾 Saving model...")
try:
    model.save_pretrained(OUTPUT_DIR)
    tokenizer.save_pretrained(OUTPUT_DIR)
    print(f"✅ Model saved to: {OUTPUT_DIR}\n")
except Exception as e:
    print(f"⚠️ Save error: {str(e)}\n")

In [ ]:
# Step 16: Evaluate
print("="*60)
print("📊 Final Evaluation")
print("="*60)
try:
    results = trainer.evaluate()
    print(f"Eval Loss: {results.get('eval_loss', 'N/A'):.4f}")
except:
    print("Evaluation skipped")

In [ ]:
# Step 17: Test generation
print("\n" + "="*60)
print("🧪 Testing Generation")
print("="*60 + "\n")

model.eval()

test_cases = [
    "Email: Hi, I need help with",
    "Email: Please find attached the quarterly report",
    "Email: Meeting scheduled for tomorrow at",
]

for i, prompt in enumerate(test_cases, 1):
    print(f"Test {i}:")
    print(f"Input: {prompt}")

    try:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=60,
                temperature=0.7,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.1,
                pad_token_id=tokenizer.eos_token_id,
            )

        generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Output: {generated}\n")

    except Exception as e:
        print(f"Generation error: {str(e)}\n")

In [ ]:
# Step 18: Compress for download
print("="*60)
print("📦 Compressing model for download")
print("="*60)

!cd /kaggle/working && tar -czf smollm2-email-finetuned.tar.gz smollm2-email-finetuned/ 2>/dev/null

print("\n✅ Compression complete!")
print("📥 Download: SmolLM3-3B-email-finetuned.tar.gz")

# Memory report
print("\n" + "="*60)
print("📊 GPU Memory Report")
print("="*60)
print(f"Allocated: {torch.cuda.memory_allocated(0) / 1024**3:.2f} GB")
print(f"Reserved: {torch.cuda.memory_reserved(0) / 1024**3:.2f} GB")
print(f"Max allocated: {torch.cuda.max_memory_allocated(0) / 1024**3:.2f} GB")

print("\n" + "="*60)
print("🎉 Fine-tuning Complete!")
print("="*60)